In [1]:
import numpy as np
import matplotlib.pyplot as plot
import pandas as pd
import datetime as dt
from salishsea_tools import evaltools as et

%matplotlib inline

In [30]:
# path to model files:
PATH= '/results/SalishSea/nowcast-green.201812/'

# start and end dates for analysis:
start_date = dt.datetime(2015,1,1)
end_date = dt.datetime(2015,3,1)

# number of days per model file:
flen=1

# dictionary mapping desired model variables to the file types where they are found
filemap={'diatoms':'ptrc_T','ciliates':'ptrc_T',
         'flagellates':'ptrc_T','vosaline':'grid_T','votemper':'grid_T'}

# dictionary mapping model file types to their time resolution in hours (1 is hourly files, 24 is daily)
fdict={'ptrc_T':1,'grid_T':1}

# results format
# -- nowcast: files like 01jan15/SalishSea_1h_20150101_20150101_ptrc_T.nc
# -- long: files like SalishSea_1h_20150206_20150804_ptrc_T_20150427-20150506.nc, all in one directory
namfmt='nowcast'

In [3]:
df1=et.loadDFOCTD(datelims=(start_date,end_date))
print(len(df1))
df1.head()

5419


,Year,Month,Day,Hour,Lat,Lon,Z,SA,CT,dtUTC
0,2015.0,2.0,11.0,5.296111,48.613333,-123.243833,1.983099,29.849611,8.942753,2015-02-11 05:17:46
1,2015.0,2.0,11.0,5.296111,48.613333,-123.243833,2.974641,29.852177,8.942511,2015-02-11 05:17:46
2,2015.0,2.0,11.0,5.296111,48.613333,-123.243833,3.966178,29.869012,8.941135,2015-02-11 05:17:46
3,2015.0,2.0,11.0,5.296111,48.613333,-123.243833,4.957711,29.915690,8.936661,2015-02-11 05:17:46
4,2015.0,2.0,11.0,5.296111,48.613333,-123.243833,5.949239,29.913935,8.936489,2015-02-11 05:17:46


In [4]:
        from sqlalchemy import create_engine, case
        from sqlalchemy.orm import create_session
        from sqlalchemy.ext.automap import automap_base
        from sqlalchemy.sql import and_, or_, not_, func

In [5]:
basedir='/ocean/shared/SalishSeaCastData/DFO/CTD/'
dbname='DFO_CTD.sqlite'
datelims=(start_date,end_date)

In [6]:
    engine = create_engine('sqlite:///' + basedir + dbname, echo = False)
    Base = automap_base()
    # reflect the tables in salish.sqlite:
    Base.prepare(engine, reflect=True)
    # mapped classes have been created
    # existing tables:
    StationTBL=Base.classes.StationTBL
    ObsTBL=Base.classes.ObsTBL
    CalcsTBL=Base.classes.CalcsTBL
    session = create_session(bind = engine, autocommit = False, autoflush = True)
    SA=case([(CalcsTBL.Salinity_T0_C0_SA!=None, CalcsTBL.Salinity_T0_C0_SA)], else_=
             case([(CalcsTBL.Salinity_T1_C1_SA!=None, CalcsTBL.Salinity_T1_C1_SA)], else_=
             case([(CalcsTBL.Salinity_SA!=None, CalcsTBL.Salinity_SA)], else_= None)))
    CT=case([(CalcsTBL.Temperature_Primary_CT!=None, CalcsTBL.Temperature_Primary_CT)], else_=
             case([(CalcsTBL.Temperature_Secondary_CT!=None, CalcsTBL.Temperature_Secondary_CT)], else_=CalcsTBL.Temperature_CT))
    ZD=case([(ObsTBL.Depth!=None,ObsTBL.Depth)], else_= CalcsTBL.Z)


In [24]:
Fl=case([(ObsTBL.Fluorescence_URU_Wetlabs!=None, ObsTBL.Fluorescence_URU_Wetlabs)], else_= ObsTBL.Fluorescence_URU_Seapoint)
Fl_units=case([(ObsTBL.Fluorescence_URU_Wetlabs!=None, ObsTBL.Fluorescence_URU_Wetlabs_units)], else_= ObsTBL.Fluorescence_URU_Seapoint)


In [25]:
start_y=datelims[0].year
start_m=datelims[0].month
start_d=datelims[0].day
end_y=datelims[1].year
end_m=datelims[1].month
end_d=datelims[1].day

In [28]:
qry=session.query(StationTBL.StartYear.label('Year'),StationTBL.StartMonth.label('Month'),
              StationTBL.StartDay.label('Day'),StationTBL.StartHour.label('Hour'),
              StationTBL.Lat,StationTBL.Lon,ZD.label('Z'),SA.label('SA'),CT.label('CT'),
                  Fl.label('Fl'),Fl_units.label('Fl_units')).\
        select_from(StationTBL).join(ObsTBL,ObsTBL.StationTBLID==StationTBL.ID).\
        join(CalcsTBL,CalcsTBL.ObsTBLID==ObsTBL.ID).filter(and_(Fl>=0,
                                                                or_(StationTBL.StartYear>start_y,
                                                                 and_(StationTBL.StartYear==start_y, StationTBL.StartMonth>start_m),
                                                                 and_(StationTBL.StartYear==start_y, StationTBL.StartMonth==start_m, StationTBL.StartDay>=start_d)),
                                                                or_(StationTBL.StartYear<end_y,
                                                                 and_(StationTBL.StartYear==end_y,StationTBL.StartMonth<end_m),
                                                                 and_(StationTBL.StartYear==end_y,StationTBL.StartMonth==end_m, StationTBL.StartDay<=end_d)),
                                                            StationTBL.Lat>47-3/2.5*(StationTBL.Lon+123.5),
                                                            StationTBL.Lat<47-3/2.5*(StationTBL.Lon+121),
                                                            StationTBL.Include==True,ObsTBL.Include==True,CalcsTBL.Include==True))
df1=pd.DataFrame(qry.all())
df1['dtUTC']=[dt.datetime(int(y),int(m),int(d))+dt.timedelta(hours=h) for y,m,d,h in zip(df1['Year'],df1['Month'],df1['Day'],df1['Hour'])]

In [29]:
df1

,Year,Month,Day,Hour,Lat,Lon,Z,SA,CT,Fl,Fl_units,dtUTC
0,2015.0,2.0,11.0,5.296111,48.613333,-123.243833,1.983099,29.849611,8.942753,0.504,0.504,2015-02-11 05:17:46
1,2015.0,2.0,11.0,5.296111,48.613333,-123.243833,2.974641,29.852177,8.942511,0.500,0.500,2015-02-11 05:17:46
2,2015.0,2.0,11.0,5.296111,48.613333,-123.243833,3.966178,29.869012,8.941135,0.491,0.491,2015-02-11 05:17:46
3,2015.0,2.0,11.0,5.296111,48.613333,-123.243833,4.957711,29.915690,8.936661,0.482,0.482,2015-02-11 05:17:46
4,2015.0,2.0,11.0,5.296111,48.613333,-123.243833,5.949239,29.913935,8.936489,0.490,0.490,2015-02-11 05:17:46
...,...,...,...,...,...,...,...,...,...,...,...,...
389,2015.0,2.0,11.0,11.068611,48.300833,-124.000333,173.453346,33.257358,8.676207,0.108,0.108,2015-02-11 11:04:07
390,2015.0,2.0,11.0,11.068611,48.300833,-124.000333,174.444086,33.288429,8.656075,0.103,0.103,2015-02-11 11:04:07
391,2015.0,2.0,11.0,11.068611,48.300833,-124.000333,175.434821,33.325134,8.628147,0.105,0.105,2015-02-11 11:04:07
392,2015.0,2.0,11.0,11.068611,48.300833,-124.000333,176.425551,33.340536,8.616082,0.113,0.113,2015-02-11 11:04:07


In [31]:
data=et.matchData(data=df1,filemap=filemap, fdict=fdict, mod_start=start_date, mod_end=end_date, 
                  mod_nam_fmt=namfmt, mod_basedir=PATH, mod_flen=flen)